In [40]:
import time
def fast_overlaps(a,b,min_length=3):
    if len(b)< min_length:
        return 0, 'string b is too short!'
    if len(a) < min_length:
        return 0, 'string a is too short!'
    start = 0
    while len(a[start:]) >= min_length:
        if b.startswith(a[start:]):
            return len(a[start:])
        start += 1
    return -1, 'no overlap'    

def readFastq(filename):
    sequences=[]
    qualities=[]
    with open(filename,'r') as fh:
        while True:
            fh.readline()
            seq=fh.readline().rstrip()
            fh.readline()
            qual=fh.readline().rstrip()
            if len(seq)==0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

def indexSuffix(reads, k=30):
    suffx_dic={}
    for read in reads:
        suffix=read[-k:] #get the suffix of length k
        #add the suffix as a key to the dictionary
        suffx_dic[suffix]=set()
        #populate with the corresponding kmers
    for i in range(len(reads)):
        read = reads[i]
        for j in range (len(read)-k+1): #remove '+1' if you want to avoid to add the suffix itself
            kmer=read[j:j+k]
            if kmer in suffx_dic:
                suffx_dic[kmer].add(i)
    return suffx_dic

In [2]:
reads,qualities=readFastq("/Users/venturaa/Google Drive/Computation files/ADS_course/ERR266411_1.for_asm.fastq")

In [50]:
def official_overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    # if len(b)< min_length:
    #    return 0
    #if len(a) < min_length:
    #    return 0
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's suffx in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

In [51]:
def overlap_graph(reads, k=30):
    dic = indexSuffix(reads, k)  # first create the suffix dictionary and populate.
    ''' each key in dic is the suffix of at least one read. to each key is associated a set of indexes
    pointing to the reads in which a kmer is identical to the key. this way, given a suffix it is easy to know in which
    reads to look for an overlap
    and which reads can be ignored.'''

    #create a list in which each index corresponds to a read, and the associated set corresponds to the overlapping reads
    graph = []
    read_index=0
    for read in reads:
        overlaps=[]
        suffix=read[-k:]
        candidate_reads=dic[suffix] #indexes of the reads that contain the sequence corresponding to the suffix
        for candidate in candidate_reads:
            if read_index != candidate:   #to avoid finding overlaps to itself!
                hit=official_overlap(read, reads[candidate], k)
                if hit > 0:
                    overlaps.append((candidate, hit)) #tuple containing the index of the read and the length of the match
                    #print 'overlap found between: \nread %d %s\nread %d %s' %(read_index,reads[read_index],candidate,reads[candidate])
        graph.append(overlaps) #add to the graph the overlaps found (if any) for the read examined
        read_index += 1
    return graph


In [54]:
def my_overlap_graph(reads, k=30):
    dic = indexSuffix(reads, k)  # first create the suffix dictionary and populate.
    ''' each key in dic is the suffix of at least one read. to each key is associated a set of indexes
    pointing to the reads in which a kmer is identical to the key. this way, given a suffix it is easy to know in which
    reads to look for an overlap
    and which reads can be ignored.'''

    #create a list in which each index corresponds to a read, and the associated set corresponds to the overlapping reads
    graph = []
    read_index=0
    for read in reads:
        overlaps=[]
        suffix=read[-k:]
        candidate_reads=dic[suffix] #indexes of the reads that contain the sequence corresponding to the suffix
        for candidate in candidate_reads:
            hit=fast_overlaps(read, reads[candidate], k)
            if hit >= 0 & hit != len(read): #to avoid matches to same read
                 overlaps.append((candidate, hit)) #tuple containing the index of the read and the length of the match
                #print 'overlap found between: \nread %d %s\nread %d %s' %(read_index,reads[read_index],candidate,reads[candidate])
        graph.append(overlaps) #add to the graph the overlaps found (if any) for the read examined
        read_index += 1
    return graph

In [55]:
t0=time.time()
overlap=overlap_graph(reads,30)
t1=time.time()-t0
print overlap[0:3], t1

[[(2, 98), (11, 99)], [], []] 9.72167897224


In [56]:
len(overlap)

10000

In [61]:
#calculate number of reads overlaps (number of edges)
edges=[]
for node in overlap:
    edges.append(len(node))
print sum(edges)

904746


In [63]:
#calculate the number of nodes with at least 1 edge
with_edges = 0
no_edges =0
for edge in edges:
    if edge > 0:
        with_edges += 1
    else: 
        no_edges += 1

In [67]:
print 'reads with edges:\t{} \nreads without edges:\t{}'.format(with_edges, no_edges)

reads with edges:	7161 
reads without edges:	2839
